# Notebook 3 : Création de la base de données Vectorielle

L'objectif de ce notebook est de créer une base de données vectorielle à partir des données de notre fichier `cocktails.csv`.

Nous allons enregistrer dans la base de donnée toutes les colonnes, et en plus nous allons enregistrer une colonne `vector` qui contiendra un vecteur de dimension 384 construit à partir des colonnes `description`, `tags`, `ingredients` et `about`.

**Commande SQl pour créer la table :**

```sql
CREATE TABLE cocktails (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    img_link TEXT,
    description TEXT,
    tags TEXT,
    ingredients TEXT,
    recipe TEXT,
    about TEXT,
    vector BLOB
);
```

In [29]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

def get_embedding(text):
    return model.encode(text)

In [ ]:
import pandas as pd

cocktails = pd.read_csv('../data/cocktails.csv').fillna('')

In [ ]:
import sqlite3

conn = sqlite3.connect('database.db')
cur = conn.cursor()

for i, row in cocktails.iterrows():
    line_to_embed = str(
        row['description'] +
        '\n\n' +
        'Liste des tags:\n' + '\n'.join(row['tags'].split("; ")) +
        '\n\n' +
        'Liste des ingrédients:\n' + '\n'.join(row['ingredients'].split("; ")) +
        '\n\n' +
        'A propos:\n' + row['about']
    )

    cur.execute(
        'INSERT INTO cocktails (name, img_link, description, tags, ingredients, recipe, about, vector) VALUES (?, ?, ?, ?, ?, ?, ?, ?)',
        (
            row['name'],
            row['img_link'],
            row['description'],
            row['tags'],
            row['ingredients'],
            row['recipe'],
            row['about'],
            sqlite3.Binary(get_embedding(line_to_embed).tobytes())
        )
    )

conn.commit()

In [33]:
cur.execute('SELECT * FROM cocktails')
for row in cur.fetchall():
    print(row)

(2131, 'Rainbow cocktail', '/media/k2/items/cache/1519e954ae6bd629544356cae3e51766_S.jpg', "Le Rainbow Cocktail : une recette aux couleurs de l'arc-en-ciel.", 'Direct au verre; Jus d’orange; Sirop de grenadine; Vodka; Curacao bleu', "3 cl de vodka; 2 cl de Curaçao Bleu; 8 cl de jus d'orange; 2 cl de sirop de grenadine", "Mélanger dans un petit recipient ou verre la vodka et le curaçao bleu, mettre ce mélange de côté pour plus tard.; Remplir le verre de glaçons.; Verser dans le verre le sirop de grenadine.; Verser le jus d'orange (la présence de glaçons va permettre d'étager les ingrédients).; Verser le mélange vodka/curacao préparé à l'étape 1.", "Rainbow Cocktail signifie Cocktail Arc-en-ciel, il s'agit de la version simplifiée que nous vous proposons ici car vous pouvez aussi ajouter du jus d'ananas au dessus du jus d'orange.", b'Z\xc0\xdd\xba\xbb7\t\xbdz\xf0\xc0\xbc\x81\xa1\xa0\xbd\x08e\x9f\xbc\x17\xf5\x8d=\xaf\xc7\xe8=#\x96\x80<\xf8\xad\xc3=y\xa3\x9e\xbd\xbba\xee\xbc\xc4\xf7[\xbd$-

In [34]:
conn.close()